In [14]:
import pandas as pd
import numpy as np

In [15]:
data = pd.read_csv("../data/data_not_clean.csv")
data = data[["Id", "Title", "Body", "Tags"]]
data.reset_index(inplace=True)
data.drop(columns='Id', inplace=True)

data.head()

,index,Title,Body,Tags
0,0,Unable to continue with rebase due to untracke...,<p>I'm currently mergin two branches using <co...,<git>
1,1,Progressive Web App on iOS 12.2 stuck in offli...,<p>I installed a custom progressive web app vi...,<ios><progressive-web-apps>
2,2,__PRETTY_FUNCTION__ in constant expression,<p>Please refer to this snippet:</p>\n\n<pre><...,<c++><c++17><constant-expression>
3,3,How to fix 'http: named cookie not present' in...,<p>I'm building a small dinner/plan management...,<http><go><cookies><jwt><postman>
4,4,How can I read a file which will be upload fro...,<p>I create a method in my .Net Core API which...,<c#><file><upload><asp.net-core-webapi>
